# 7.7 Hannan-Rissanen

Start the same way, linear regression to get the AR coeffs, but then going to count the AR and MA together as the same thing  
Simpler algorithm the the previous one  

**HR is an innovation regression method of ARMA madeling**



In [1]:
import pandas as pd
import numpy as np 
df = pd.read_csv('./data/UK_job_vacancies.csv')

# Design matrix is X || Target vector is Y
def linear_regression(design_matrix, target_vector):
    return np.linalg.inv(design_matrix.transpose() @ design_matrix) @design_matrix.transpose() @ target_vector

# Use the LR to get the AR coeffs - think that's just broken out of original train_ar formula used
def train_ar(values, order):
    target_vector = np.array(values[order:])
    lagged_values = []
    for i in range(len(values) - order):
        lagged_values.append(values[i:i+order])
    design_matrix = np.array(lagged_values)
    return linear_regression(design_matrix, target_vector)

In [2]:
series = df['Vacancies']
ar_deg = 3
ma_deg = 2


Before kept the AR coeffs and the MA coeffs seperate, new we put them together

Now we just use the above code to get the AR coeﬀicients. We will then put them together with
some 0s that will be place-holders for MA coeﬀicients.

In [3]:
# 3 AR coefs - 2 MA Coefs
coef = np.append(train_ar(series, ar_deg), np.zeros(ma_deg))
print('Coefs:', coef)

Coefs: [ 0.34228651 -1.34176882  2.00015731  0.          0.        ]


Have the initial value for the AR coeﬀicients.   
Now, as in the gradient descent method, we have an outer loop with x as our loop variable.   
Each time we iterate that loop, we produce new coeﬀicients and new root mean square errors.  
The coeﬀicients are computed by a liner regression

Coefs comupted by a LR:  
```coef = linear_regression(design_matrix, series[max(ar_deg, ma_deg):])```

In [4]:
errors = [0.0] * len(series)

for x in range(100):

    # Compute Design Matrix
    design_matrix_rows = []

    for i in range(max(ar_deg, ma_deg), len(series)):
        # make values - take lags and errors
        # 3 from series (AR(3)) and 2 from errors (MA(2))
        values = np.append(series[i-ar_deg:i], errors[i-ma_deg:i])
        
        # predictor - dotting the beta coefs
        # coefs for both AR and MA here, get linear combination
        pred = np.dot(values, coef)

        # add values to design matrix
        design_matrix_rows.append(values)

        # compute errors
        errors[i] = series[i] - pred
    
    design_matrix = np.array(design_matrix_rows)

    # get coefs using LR on design matrix and observed target values ('vacancies')
    coef = linear_regression(design_matrix, series[max(ar_deg, ma_deg):])

    # calculate RMSE
    rmse = np.sqrt(np.dot(errors, errors)/(len(errors) - max(ar_deg, ma_deg)))

    if (x % 10) == 0:
        print("step", x, ":", rmse, coef)

step 0 : 13.340383927080804 [-0.29278315  0.60446485  0.69029285  0.88716733  1.30446452]
step 10 : 12.04183649316871 [ 0.05999143 -0.3733167   1.31473426  0.70885985  0.73688165]
step 20 : 12.054377492074284 [ 0.05959156 -0.37588319  1.3176913   0.70421973  0.73290753]
step 30 : 12.054323451755405 [ 0.05973176 -0.37622555  1.3178933   0.7041793   0.73271694]
step 40 : 12.054308706319706 [ 0.05973228 -0.37622322  1.31789046  0.70418429  0.732721  ]
step 50 : 12.054308718307373 [ 0.05973213 -0.37622284  1.31789023  0.70418435  0.73272122]
step 60 : 12.054308734288389 [ 0.05973212 -0.37622285  1.31789023  0.70418435  0.73272122]
step 70 : 12.05430873433979 [ 0.05973212 -0.37622285  1.31789023  0.70418435  0.73272121]
step 80 : 12.054308734325556 [ 0.05973212 -0.37622285  1.31789023  0.70418435  0.73272121]
step 90 : 12.054308734305945 [ 0.05973212 -0.37622285  1.31789023  0.70418435  0.73272121]


In [5]:
print(design_matrix)

[[567.         563.         553.           0.           0.        ]
 [563.         553.         552.           0.           1.15204979]
 [553.         552.         535.           1.15204979 -18.89749126]
 ...
 [568.         664.         762.          47.00296232   7.09816489]
 [664.         762.         846.           7.09816489  19.35204046]
 [762.         846.         915.          19.35204046  27.90647509]]


Each step gives RMSE, now below 13 where ARMA with GD was higher at 13 so this is better.  

Then get the AR and MA coefs, can see they stabalize. 

**Linear regression on steroids**  
Do LR, make design matrix, do LR, make design matrix and repeat

*Usually HR models use bigger AR model for initial step*

# 7.8 Hannen-Rissanen Diffs

### Bringing in the "I" in ARIMA
I stands for Integration, really about differences

[Notes]('https://www.machinelearningplus.com/time-series/augmented-dickey-fuller-test/')

In [6]:
import matplotlib
import math

jobs_df = pd.read_csv('./data/UK_job_vacancies.csv')

# set AR deg to a larger magnitude - 20

series = jobs_df['Vacancies']

ar_deg = 10

ma_deg = 2

coef = np.append(train_ar(series, ar_deg), np.zeros(ma_deg))
print(coef)
# coef = [-0.0941, -0.1832, 1.2754, 0.5262, 0.7267] # coefs from statsmodels
errors = [0.0] * len(series)

for x in range(20):
    design_matrix_rows = []
    for i in range(max(ar_deg, ma_deg), len(series)):
        values = np.append(series[i-ar_deg:i], errors[i-ma_deg:i])
        pred = np.dot(values, coef)
        design_matrix_rows.append(values)
        errors[i] = series[i] - pred
        
    design_matrix = np.array(design_matrix_rows)
    coef = linear_regression(design_matrix, series[max(ar_deg, ma_deg):])

print("step", x, ":", math.sqrt(np.dot(errors, errors)/(len(errors) - max(ar_deg, ma_deg))))
print(coef) # 12.0543


[ 0.04289748 -0.12204264 -0.13419879  0.60114682 -0.54379875 -0.29137531
  1.05463138 -0.50054911 -1.13997554  2.03387541  0.          0.        ]
step 19 : 9.107590717940694e+60
[ 3.17344572e+02 -6.43771588e+02  1.73437306e+02  6.90726809e+02
 -3.25780348e+02 -1.67960576e+03  3.33672663e+03 -2.82343339e+03
  9.59873640e+02 -3.63829133e+00  2.06873163e-56  1.18113673e-56]


Decide whether have something stationary or not  

Will use standard algorithms in stats models. **Augmented Dickey-Fuller**

In [7]:
from statsmodels.tsa.stattools import adfuller
jobs_adf = adfuller(series)
print(jobs_adf)
print('p-value', jobs_adf[1])

(-0.8721852659607139, 0.7969941749563074, 15, 228, {'1%': -3.4593607492757554, '5%': -2.8743015807562924, '10%': -2.5735714042782396}, 1783.501660881836)
p-value 0.7969941749563074


If p-value < 0.05 = stationary  
If p-value > 0.05 = non-stationary

P-value here is 0.79 so series is non-stationary  

Now that established it's not stationary - now take the H-R diffs

In [8]:
diffs = (series - series.shift()).dropna().reset_index()['Vacancies']
print('p-value', adfuller(diffs)[1])

p-value 0.0010897388496611759


P-value < 0.05 so now have stationary data after taking diffs  

Now do HR on the diffs

In [9]:
ar_deg= 20
ma_deg = 3
coef = np.append(train_ar(diffs, ar_deg), np.zeros(ma_deg))

for x in range(40):
    design_matrix_rows = []
    errors = np.zeros(len(diffs))

    for i in range(max(ar_deg, ma_deg), len(diffs)):
        values = np.append(diffs[i-ar_deg:i], errors[i-ma_deg:i])
        pred = np.dot(values, coef)
        design_matrix_rows.append(values)
        errors[i] = diffs[i] - pred

    design_matrix = np.array(design_matrix_rows)

    coef = linear_regression(design_matrix, diffs[max(ar_deg, ma_deg):])
    if (x % 10) == 0:
        print("step", x, ":", math.sqrt(np.dot(errors, errors)/(len(errors) - max(ar_deg, ma_deg))))

step 0 : 11.103606112877063
step 10 : 11.083643627580475
step 20 : 11.083535989361344
step 30 : 11.083529332042765
